In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zillow-prize-1/properties_2017.csv
/kaggle/input/zillow-prize-1/sample_submission.csv
/kaggle/input/zillow-prize-1/zillow_data_dictionary.xlsx
/kaggle/input/zillow-prize-1/train_2016_v2.csv
/kaggle/input/zillow-prize-1/properties_2016.csv
/kaggle/input/zillow-prize-1/train_2017.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')
import gc

print("데이터 로드 중...")
train_2016 = pd.read_csv('/kaggle/input/zillow-prize-1/train_2016_v2.csv')
train_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/train_2017.csv')
properties_2016 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2016.csv')
properties_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2017.csv')
sample_submission = pd.read_csv('/kaggle/input/zillow-prize-1/sample_submission.csv')

데이터 로드 중...


In [3]:
properties_2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [4]:
print(properties_2016.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      int64  
 1   airconditioningtypeid         float64
 2   architecturalstyletypeid      float64
 3   basementsqft                  float64
 4   bathroomcnt                   float64
 5   bedroomcnt                    float64
 6   buildingclasstypeid           float64
 7   buildingqualitytypeid         float64
 8   calculatedbathnbr             float64
 9   decktypeid                    float64
 10  finishedfloor1squarefeet      float64
 11  calculatedfinishedsquarefeet  float64
 12  finishedsquarefeet12          float64
 13  finishedsquarefeet13          float64
 14  finishedsquarefeet15          float64
 15  finishedsquarefeet50          float64
 16  finishedsquarefeet6           float64
 17  fips                          float64
 18  fireplacecnt          

In [5]:
print(properties_2016.describe())

           parcelid  airconditioningtypeid  architecturalstyletypeid  \
count  2.985217e+06          811519.000000               6061.000000   
mean   1.332586e+07               1.931166                  7.202607   
std    7.909966e+06               3.148587                  2.436290   
min    1.071172e+07               1.000000                  2.000000   
25%    1.164371e+07               1.000000                  7.000000   
50%    1.254509e+07               1.000000                  7.000000   
75%    1.409712e+07               1.000000                  7.000000   
max    1.696019e+08              13.000000                 27.000000   

       basementsqft   bathroomcnt    bedroomcnt  buildingclasstypeid  \
count   1628.000000  2.973755e+06  2.973767e+06         12629.000000   
mean     646.883292  2.209143e+00  3.088949e+00             3.725948   
std      538.793473  1.077754e+00  1.275859e+00             0.501700   
min       20.000000  0.000000e+00  0.000000e+00             1.0

In [6]:
print(properties_2016["airconditioningtypeid"].value_counts())

airconditioningtypeid
1.0     742364
13.0     58457
5.0       8795
11.0      1818
12.0        59
9.0         19
3.0          7
Name: count, dtype: int64


In [7]:
# Checking the Number of missing Values
print(properties_2016.isnull().sum())

parcelid                              0
airconditioningtypeid           2173698
architecturalstyletypeid        2979156
basementsqft                    2983589
bathroomcnt                       11462
bedroomcnt                        11450
buildingclasstypeid             2972588
buildingqualitytypeid           1046729
calculatedbathnbr                128912
decktypeid                      2968121
finishedfloor1squarefeet        2782500
calculatedfinishedsquarefeet      55565
finishedsquarefeet12             276033
finishedsquarefeet13            2977545
finishedsquarefeet15            2794419
finishedsquarefeet50            2782500
finishedsquarefeet6             2963216
fips                              11437
fireplacecnt                    2672580
fullbathcnt                      128912
garagecarcnt                    2101950
garagetotalsqft                 2101950
hashottuborspa                  2916203
heatingorsystemtypeid           1178816
latitude                          11437


In [8]:
train_2016.tail()

,parcelid,logerror,transactiondate
90270,10774160,-0.0356,2016-12-30
90271,12046695,0.0070,2016-12-30
90272,12995401,-0.2679,2016-12-30
90273,11402105,0.0602,2016-12-30
90274,12566293,0.4207,2016-12-30


In [9]:
# train data merge
train = pd.concat([train_2016,train_2017],axis=0)
print(train.shape)

(167888, 3)


**Temporal Priority of Real Estate Data (Recency):** The characteristics of real estate can change over time due to remodeling, tax changes, ownership changes, etc. Therefore, if data from 2017 exists, it is generally considered more recent and prioritized for use.

**Statistical Perspective: Efficiency in Handling Missing Values:** Useful auxiliary data is used to fill in missing values. This approach is more likely to maintain values closer to actual data than simply replacing missing values with mean or median values.

**the private leaderboard will be evaluated based on data from October 15, 2017, to December 15, 2017** (according to the competition description, the "sales tracking period" uses 2017 transaction data)

In [10]:
# Merging properties_2016 and properties_2017
properties = properties_2017.copy()
for col in properties_2017.columns:
    properties[col] = properties_2017[col].fillna(properties_2016[col])

In [11]:
# Merging training data and real estimate data
train_df = train.merge(properties, on ='parcelid', how = 'left')
print(f"병합된 훈련 데이터 크기: {train_df.shape}")

병합된 훈련 데이터 크기: (167888, 60)


In [12]:
train_df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,8.0,...,NaN,NaN,218000.0,545000.0,2016.0,327000.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,1.0,NaN,NaN,3.5,4.0,NaN,4.0,...,NaN,NaN,346458.0,974900.0,2016.0,628442.0,10153.02,NaN,NaN,6.037800e+13
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,8.0,...,NaN,NaN,197900.0,989500.0,2016.0,791600.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,7.0,...,NaN,NaN,174133.0,248613.0,2016.0,74480.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,7.0,...,2.0,NaN,172160.0,441177.0,2016.0,269017.0,5488.96,NaN,NaN,6.059042e+13


In [13]:
print(train_df.isnull().sum())

parcelid                             0
logerror                             0
transactiondate                      0
airconditioningtypeid            82523
architecturalstyletypeid        167103
basementsqft                    167699
bathroomcnt                         51
bedroomcnt                          51
buildingclasstypeid             167101
buildingqualitytypeid            18167
calculatedbathnbr                  115
decktypeid                      165659
finishedfloor1squarefeet        144202
calculatedfinishedsquarefeet        60
finishedsquarefeet12               751
finishedsquarefeet13            167380
finishedsquarefeet15            150416
finishedsquarefeet50            144202
finishedsquarefeet6             165971
fips                                51
fireplacecnt                    134656
fullbathcnt                        115
garagecarcnt                     78582
garagetotalsqft                  78582
hashottuborspa                  161025
heatingorsystemtypeid    

In [14]:
print(len(train_df))
important_features = [
    'latitude', 'longitude', 'lotsizesquarefeet', 'calculatedfinishedsquarefeet',
    'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
    'finishedsquarefeet50', 'finishedsquarefeet6',
    'bathroomcnt', 'bedroomcnt', 'yearbuilt']

# 3. Remove only non-important columns with more than 50% missing values.
# 3. 중요하지 않은 피처 중 결측치 비율이 50% 이상인 컬럼 제거
missing_ratio = train_df.isnull().sum() / len(train_df) * 100
cols_to_drop = missing_ratio[(missing_ratio > 50) & (~missing_ratio.index.isin(important_features))].index
train_df_filtered = train_df.drop(columns=cols_to_drop)

print(train_df_filtered.isnull().sum())

167888
parcelid                             0
logerror                             0
transactiondate                      0
airconditioningtypeid            82523
bathroomcnt                         51
bedroomcnt                          51
buildingqualitytypeid            18167
calculatedbathnbr                  115
calculatedfinishedsquarefeet        60
finishedsquarefeet12               751
finishedsquarefeet13            167380
finishedsquarefeet15            150416
finishedsquarefeet50            144202
finishedsquarefeet6             165971
fips                                51
fullbathcnt                        115
garagecarcnt                     78582
garagetotalsqft                  78582
heatingorsystemtypeid            22109
latitude                            51
longitude                           51
lotsizesquarefeet                 1658
propertycountylandusecode           51
propertylandusetypeid               51
propertyzoningdesc               16718
rawcensustractandb

In [15]:
# Remove less relevant features (based on real estate perspective)
# 관련성 낮은 피처 제거 (부동산 관점에서 판단)
irrelevant_cols = ['unitcnt', 'rawcensustractandblock', 'fips'] # 예시
train_df_clean = train_df_filtered.drop(irrelevant_cols, errors= 'ignore')

In [16]:
train_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167888 entries, 0 to 167887
Data columns (total 39 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      167888 non-null  int64  
 1   logerror                      167888 non-null  float64
 2   transactiondate               167888 non-null  object 
 3   airconditioningtypeid         85365 non-null   float64
 4   bathroomcnt                   167837 non-null  float64
 5   bedroomcnt                    167837 non-null  float64
 6   buildingqualitytypeid         149721 non-null  float64
 7   calculatedbathnbr             167773 non-null  float64
 8   calculatedfinishedsquarefeet  167828 non-null  float64
 9   finishedsquarefeet12          167137 non-null  float64
 10  finishedsquarefeet13          508 non-null     float64
 11  finishedsquarefeet15          17472 non-null   float64
 12  finishedsquarefeet50          23686 non-null

In [17]:
print(train_df_clean)

        parcelid  logerror transactiondate  airconditioningtypeid  \
0       11016594  0.027600      2016-01-01                    1.0   
1       14366692 -0.168400      2016-01-01                    1.0   
2       12098116 -0.004000      2016-01-01                    1.0   
3       12643413  0.021800      2016-01-02                    1.0   
4       14432541 -0.005000      2016-01-02                    NaN   
...          ...       ...             ...                    ...   
167883  10833991 -0.002245      2017-09-20                    1.0   
167884  11000655  0.020615      2017-09-20                    NaN   
167885  17239384  0.013209      2017-09-21                    1.0   
167886  12773139  0.037129      2017-09-21                    1.0   
167887  12826780  0.007204      2017-09-25                    NaN   

        bathroomcnt  bedroomcnt  buildingqualitytypeid  calculatedbathnbr  \
0               2.0         3.0                    8.0                2.0   
1               3

In [18]:
# 1.범주형 피처 원-핫 인코딩
categorical_cols = ['airconditioningtypeid', 'heatingorsystemtypeid', 
                   'propertycountylandusecode', 'propertylandusetypeid', 
                    'propertyzoningdesc']
# drop_first -> Prevent multicollinearity by removing the first category.
"""If it's information everyone can already access, 
there's no need to include it! It reduces redundancy."""

train_df_encoded = pd.get_dummies(train_df_clean, columns=categorical_cols, drop_first=True)

# Remove ID Feateature 
train_df_encoded = train_df_encoded.drop(columns =['parcelid'], errors= 'ignore')

# 3.날짜 처리 (transactiondate를 숫자형으로 변환)

"""transactiondate를 **문자열에서 날짜 타입(datetime)**으로 변환"""
train_df_encoded['transactiondate'] = pd.to_datetime(train_df_encoded['transactiondate'])
train_df_encoded['transaction_year'] = train_df_encoded['transactiondate'].dt.year
train_df_encoded['transaction_month'] = train_df_encoded['transactiondate'].dt.month
train_df_encoded['transaction_day'] = train_df_encoded['transactiondate'].dt.day
train_df_encoded = train_df_encoded.drop(columns = ['transactiondate'])


train_df_encoded.info()                   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167888 entries, 0 to 167887
Columns: 2726 entries, logerror to transaction_day
dtypes: bool(2691), float64(32), int32(3)
memory usage: 473.8 MB


In [19]:
# 숫자형 데이터만으로 상관계수 계산 준비
numeric_cols = train_df_encoded.select_dtypes(include=['int32','float64']).columns
train_df_numeric = train_df_encoded[numeric_cols]

# 중복 피쳐 제거(상관 계수로 확인 가능)
corr_matrix = train_df_numeric.corr()
high_corr_pairs = []
for col1 in corr_matrix.columns:
    for col2 in corr_matrix.columns:
        if col1 < col2 and abs(corr_matrix.loc[col1,col2]) > 0.9:
            high_corr_pairs.append((col1, col2))
X = train_df_numeric.drop(columns=['logerror'],errors='ignore') # Exclude the target.
Y = train_df_numeric['logerror']
rf = RandomForestRegressor(n_estimators=100,random_state=42,n_jobs=-1) # n_jobs: using the number of CPU core.
                                                                       # -1 : use all core on CPU
rf.fit(X,Y)

feature_importance = pd.DataFrame({
    'feature' : X.columns,
    'importance' : rf.feature_importances_
}).sort_values('importance', ascending=False)

final_cols = train_df_numeric.columns.tolist()

for col1,col2 in high_corr_pairs:
    imp1 = feature_importance[feature_importance['feature'] == col1]['importance'].values[0]
    imp2 = feature_importance[feature_importance['feature'] == col2]['importance'].values[0]
    if imp1 > imp2:
        final_cols.remove(col2)
    else:
        final_cols.remove(col1)

train_df_final = train_df_numeric[final_cols + 'logerror']
print("최종 유지된 피처:", final_cols)


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

While basic regression problems may be applicable, **Random Forest and Gradient Boosting models usually perform better.**

Even though the overall data seems abundant, **there is insufficient data for each parcel ID (house), making it difficult for deep learning models to train.**

**Time-series deep learning models (like LSTM, Transformer, etc.) require multiple sequential data points for the same entity**, but Zillow data mostly contains **only one or two transaction records**."